In [37]:
import numpy as np
import librosa

def wav2mfcc(file_path,max_pad_len = 11):
    wave,sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    return mfcc

In [38]:
import os
DATA_PATH = "./DeadSimpleSpeechRecognizer-master/data/"

def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0,len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [39]:
#convert into np array
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels, _, _ = get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
        np.save(label+'.npy', mfcc_vectors)

In [40]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio = 0.8, random_state = 42):
    labels, indices, _ = get_labels(DATA_PATH)
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X,x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
    assert X.shape[0] == len(y)
    return train_test_split(X,y, test_size = (1-split_ratio), random_state=random_state, shuffle=True)

In [42]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [43]:
save_data_to_array()

In [44]:
X_train, X_test, y_train, y_test = get_train_test()

In [45]:
X_train = X_train.reshape(X_train.shape[0],20,11,1)
X_test = X_test.reshape(X_test.shape[0],20,11,1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [46]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2,2), activation = 'relu', input_shape=(20,11,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    model.compile(loss = keras.losses.categorical_crossentropy,
                 optimizer = keras.optimizers.Adadelta(),
                 metrics = ['accuracy'])
    return model

In [47]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size = 100, epochs = 300, verbose = 1, validation_data = (X_test, y_test_hot))

Train on 4150 samples, validate on 1038 samples
Epoch 1/300
4150/4150 [==============================] - 1s 256us/step - loss: 7.2831 - acc: 0.4125 - val_loss: 8.4820 - val_acc: 0.3786
Epoch 2/300
4150/4150 [==============================] - 1s 154us/step - loss: 4.2264 - acc: 0.6087 - val_loss: 1.7042 - val_acc: 0.7707
Epoch 3/300
4150/4150 [==============================] - 1s 154us/step - loss: 3.0109 - acc: 0.6839 - val_loss: 2.4080 - val_acc: 0.7274
Epoch 4/300
4150/4150 [==============================] - 1s 154us/step - loss: 2.2480 - acc: 0.7395 - val_loss: 1.5048 - val_acc: 0.8044
Epoch 5/300
4150/4150 [==============================] - 1s 155us/step - loss: 1.6474 - acc: 0.7708 - val_loss: 1.3083 - val_acc: 0.8083
Epoch 6/300
4150/4150 [==============================] - 1s 155us/step - loss: 1.3477 - acc: 0.7865 - val_loss: 0.8882 - val_acc: 0.8362
Epoch 7/300
4150/4150 [==============================] - 1s 153us/step - loss: 1.1237 - acc: 0.7949 - val_loss: 0.6493 - val_acc: 

4150/4150 [==============================] - 1s 150us/step - loss: 0.2579 - acc: 0.9039 - val_loss: 0.2967 - val_acc: 0.8911
Epoch 61/300
4150/4150 [==============================] - 1s 137us/step - loss: 0.2866 - acc: 0.8959 - val_loss: 0.3191 - val_acc: 0.8892
Epoch 62/300
4150/4150 [==============================] - 1s 150us/step - loss: 0.2814 - acc: 0.8983 - val_loss: 0.3364 - val_acc: 0.8767
Epoch 63/300
4150/4150 [==============================] - 1s 158us/step - loss: 0.2800 - acc: 0.8957 - val_loss: 0.2729 - val_acc: 0.8998
Epoch 64/300
4150/4150 [==============================] - 1s 141us/step - loss: 0.2771 - acc: 0.8981 - val_loss: 0.3228 - val_acc: 0.8805
Epoch 65/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2811 - acc: 0.8957 - val_loss: 0.3251 - val_acc: 0.8671
Epoch 66/300
4150/4150 [==============================] - 1s 144us/step - loss: 0.2672 - acc: 0.9019 - val_loss: 0.2959 - val_acc: 0.8969
Epoch 67/300
4150/4150 [=======================

4150/4150 [==============================] - 1s 138us/step - loss: 0.2544 - acc: 0.9070 - val_loss: 0.3566 - val_acc: 0.8767
Epoch 120/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2481 - acc: 0.9084 - val_loss: 0.3706 - val_acc: 0.8651
Epoch 121/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2488 - acc: 0.9055 - val_loss: 0.3233 - val_acc: 0.8757
Epoch 122/300
4150/4150 [==============================] - 1s 155us/step - loss: 0.2432 - acc: 0.9120 - val_loss: 0.2915 - val_acc: 0.8969
Epoch 123/300
4150/4150 [==============================] - 1s 147us/step - loss: 0.2456 - acc: 0.9142 - val_loss: 0.2881 - val_acc: 0.8979
Epoch 124/300
4150/4150 [==============================] - 1s 150us/step - loss: 0.2310 - acc: 0.9116 - val_loss: 0.2764 - val_acc: 0.9008
Epoch 125/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2454 - acc: 0.9140 - val_loss: 0.2956 - val_acc: 0.9008
Epoch 126/300
4150/4150 [================

4150/4150 [==============================] - 1s 150us/step - loss: 0.2232 - acc: 0.9207 - val_loss: 0.3001 - val_acc: 0.8931
Epoch 179/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2384 - acc: 0.9159 - val_loss: 0.2968 - val_acc: 0.8998
Epoch 180/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2442 - acc: 0.9104 - val_loss: 0.2819 - val_acc: 0.9008
Epoch 181/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2250 - acc: 0.9161 - val_loss: 0.3397 - val_acc: 0.8892
Epoch 182/300
4150/4150 [==============================] - 1s 150us/step - loss: 0.2318 - acc: 0.9147 - val_loss: 0.3391 - val_acc: 0.8863
Epoch 183/300
4150/4150 [==============================] - 1s 151us/step - loss: 0.2270 - acc: 0.9154 - val_loss: 0.3233 - val_acc: 0.8902
Epoch 184/300
4150/4150 [==============================] - 1s 151us/step - loss: 0.2219 - acc: 0.9133 - val_loss: 0.3218 - val_acc: 0.8796
Epoch 185/300
4150/4150 [================

4150/4150 [==============================] - 1s 141us/step - loss: 0.2302 - acc: 0.9164 - val_loss: 0.2796 - val_acc: 0.9027
Epoch 238/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2407 - acc: 0.9164 - val_loss: 0.2827 - val_acc: 0.8969
Epoch 239/300
4150/4150 [==============================] - 1s 151us/step - loss: 0.2230 - acc: 0.9198 - val_loss: 0.3265 - val_acc: 0.8950
Epoch 240/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2103 - acc: 0.9239 - val_loss: 0.3278 - val_acc: 0.8728
Epoch 241/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2174 - acc: 0.9161 - val_loss: 0.3239 - val_acc: 0.8844
Epoch 242/300
4150/4150 [==============================] - 1s 150us/step - loss: 0.2273 - acc: 0.9164 - val_loss: 0.2894 - val_acc: 0.9027
Epoch 243/300
4150/4150 [==============================] - 1s 149us/step - loss: 0.2328 - acc: 0.9166 - val_loss: 0.2951 - val_acc: 0.8998
Epoch 244/300
4150/4150 [================

4150/4150 [==============================] - 1s 141us/step - loss: 0.2183 - acc: 0.9207 - val_loss: 0.2821 - val_acc: 0.8979
Epoch 297/300
4150/4150 [==============================] - 1s 151us/step - loss: 0.2155 - acc: 0.9186 - val_loss: 0.5150 - val_acc: 0.8372
Epoch 298/300
4150/4150 [==============================] - 1s 152us/step - loss: 0.1991 - acc: 0.9243 - val_loss: 0.2909 - val_acc: 0.9037
Epoch 299/300
4150/4150 [==============================] - 1s 147us/step - loss: 0.2324 - acc: 0.9130 - val_loss: 0.4083 - val_acc: 0.8651
Epoch 300/300
4150/4150 [==============================] - 1s 148us/step - loss: 0.2262 - acc: 0.9171 - val_loss: 0.2893 - val_acc: 0.9075


In [48]:
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1,20,11,1)
    return get_labels()[0][np.argmax(model.predict(sample_reshaped))]

print(predict('./DeadSimpleSpeechRecognizer-master/data/bed/016e2c6d_nohash_0.wav', model=model))

bed
